### Uses processing.py's code in a notebook for interactivity. Also attempts some contour length calculations. Doesn't yet convert to physical units.

This code generates a lot of `.txt` and `.png` files in the imgproc/ directory. They're gitignored so don't show up in GitHub, but you can view them locally.

# Setup

In [3]:
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
from ipywidgets import interact, fixed
import cv2

from imgproc_helpers import *

OUTPUT_DIR = "../../out"
NRRD1_PATH = "../../ExampleData/BCP_Dataset_2month_T1w.nrrd"
NRRD2_PATH = "../../ExampleData/IBIS_Dataset_12month_T1w.nrrd"
NRRD3_PATH = "../../ExampleData/IBIS_Dataset_NotAligned_6month_T1w.nrrd"
"""This is an intentionally misaligned image."""
NIFTI_PATH = "../../ExampleData/MicroBiome_1month_T1w.nii.gz"

NIFTI_UNITS = {
    "0": "Unknown",
    "1": "Meter (m)",
    "2": "Millimeter (mm)",
    "3": "Micron (μm)",
    "8": "Seconds (s)",
    "16": "Milliseconds (ms)",
    "24": "Microseconds (μs)",
    "32": "Hertz (Hz)",
    "40": "Parts-per-million (ppm)",
    "48": "Radians per second (rad/s)",
}
"""Maps the `xyzt_units` metadata field of a NIfTI file to physical meaning.

Based on https://brainder.org/2012/09/23/the-nifti-file-format/"""

reader = sitk.ImageFileReader()
reader.SetFileName(NIFTI_PATH)
# Not sure what this does, trying to read metadata
reader.LoadPrivateTagsOn()
image = reader.Execute()
# print(f"Origin: {image.GetOrigin()}")
# print(f"Dimensions: {image.GetSize()}")
# print(f"[Dimensions / 2]: {[(dimension - 1) / 2.0 for dimension in image.GetSize()]}")
# print(f"Spacing: {image.GetSpacing()}")
# print(f"reader.ReadImageInformation(): {reader.ReadImageInformation()}")
print(
    f'Units: {NIFTI_UNITS[reader.GetMetaData("xyzt_units")]}'
    if "xyzt_units" in reader.GetMetaDataKeys()
    else ""
)
# print(f"\n\nMetadata keys: {reader.GetMetaDataKeys}")

for key in reader.GetMetaDataKeys():
    # print(type(key))
    # keys are strings
    print(f"{key}: {reader.GetMetaData(key)}")

# str
# print(type(reader.GetMetaData("xyzt_units")))

Units: Millimeter (mm)
ITK_FileNotes: 
aux_file: 
bitpix: 16
cal_max: 0
cal_min: 0
datatype: 4
descrip: 
dim[0]: 3
dim[1]: 244
dim[2]: 292
dim[3]: 198
dim[4]: 1
dim[5]: 1
dim[6]: 1
dim[7]: 1
dim_info: 0
intent_code: 0
intent_name: 
intent_p1: 0
intent_p2: 0
intent_p3: 0
nifti_type: 1
pixdim[0]: 0
pixdim[1]: 0.79918
pixdim[2]: 0.797945
pixdim[3]: 0.80303
pixdim[4]: 0
pixdim[5]: 0
pixdim[6]: 0
pixdim[7]: 0
qfac: [UNKNOWN_PRINT_CHARACTERISTICS]

qform_code: 1
qform_code_name: NIFTI_XFORM_SCANNER_ANAT
qoffset_x: -96.1004
qoffset_y: -134.101
qoffset_z: -72.0985
qto_xyz: [UNKNOWN_PRINT_CHARACTERISTICS]

quatern_b: 0
quatern_c: 0
quatern_d: 0
scl_inter: 0
scl_slope: 1
sform_code: 0
sform_code_name: NIFTI_XFORM_UNKNOWN
slice_code: 0
slice_duration: 0
slice_end: 0
slice_start: 0
srow_x: 0 0 0 0
srow_y: 0 0 0 0
srow_z: 0 0 0 0
toffset: 0
vox_offset: 352
xyzt_units: 2
<class 'str'>


# Interactive version of `rotate_and_get_contour`

`rotate_and_get_contour` is already defined in `imgproc_helpers.py` but is redefined here to include `plt.show()` to display with `ipywidgets`. I also print out the number of detected contours.

Comment out smooth slice to avoid warnings in the output.

To be clear, this is not "production-quality" code. It's just for interactivity for testing purposes.

In [4]:
def rotate_and_get_contour_testing(
    img: sitk.Image, theta_x: int, theta_y: int, theta_z: int, slice_z: int
) -> sitk.Image:
    """Same as the function of the same name in imgproc_helpers.py but needs to be redefined here to include `plt.show()`."""

    euler_3d_transform = sitk.Euler3DTransform()
    # NOTE: This center is possibly incorrect.
    euler_3d_transform.SetCenter(
        img.TransformContinuousIndexToPhysicalPoint(
            [((dimension - 1) / 2.0) for dimension in img.GetSize()]
        )
    )
    euler_3d_transform.SetRotation(
        degrees_to_radians(theta_x),
        degrees_to_radians(theta_y),
        degrees_to_radians(theta_z),
    )
    rotated_image = sitk.Resample(img, euler_3d_transform)
    rotated_image_slice = rotated_image[:, :, slice_z]

    # The cast is necessary, otherwise get sitk::ERROR: Pixel type: 16-bit signed integer is not supported in 2D
    # However, this does throw some weird errors
    smooth_slice = sitk.GradientAnisotropicDiffusionImageFilter().Execute(
        sitk.Cast(rotated_image_slice, sitk.sitkFloat64)
    )

    otsu = sitk.OtsuThresholdImageFilter().Execute(smooth_slice)

    hole_filling = sitk.BinaryGrindPeakImageFilter().Execute(otsu)

    # BinaryGrindPeakImageFilter has inverted foreground/background 0 and 1, need to invert
    inverted = sitk.NotImageFilter().Execute(hole_filling)

    largest_component = select_largest_component(inverted)

    contour = sitk.BinaryContourImageFilter().Execute(largest_component)

    plt.imshow(np.ndarray.transpose(sitk.GetArrayFromImage(contour)))
    plt.axis("off")
    plt.show()

    # This does transpose but if we only care about # of contours, it's fine
    contours, hierarchy = cv2.findContours(
        sitk.GetArrayFromImage(contour), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
    )

    print(f"Number of contours: {len(contours)}")

    return contour

# Interactive display

In [5]:
interact(
    rotate_and_get_contour_testing,
    img=fixed(image),
    theta_x=(-180, 180),
    theta_y=(-180, 180),
    theta_z=(-180, 180),
    slice_z=(0, image.GetSize()[2] - 1),
)

interactive(children=(IntSlider(value=0, description='theta_x', max=180, min=-180), IntSlider(value=0, descrip…

<function __main__.rotate_and_get_contour_testing(img: SimpleITK.SimpleITK.Image, theta_x: int, theta_y: int, theta_z: int, slice_z: int) -> SimpleITK.SimpleITK.Image>

# Save `sitk` and `np` representations to text files for testing pixel spacing. Also save `sitk` and `np` array representations to PNG files.

Note: The underlying array seems to be correctly spaced since the PNG file generated from it looks correctly spaced. Note that when written to a file, it's incorrectly spaced because in mono font, height > width, but that doesn't matter.

In [10]:
from PIL import Image

# This is the real rotate_and_get_contour function
sitk_contour_slice = rotate_and_get_contour(image, 0, 0, 0, 0)
# show_fiji(contour)

write_sitk_slice(sitk_contour_slice, "contour_sitk.txt")

# Numpy binary 2D array. It's the transpose of contour_slice
np_contour_slice: np.ndarray = sitk.GetArrayFromImage(sitk_contour_slice)

write_numpy_array_slice_transpose(np_contour_slice, "contour_numpy.txt")

# Convert binary (0|1) array to binary (0|255) array to display as white in images
np_contour_slice_255: np.ndarray = binary_array_to_255_array(np_contour_slice)

PNG = Image.fromarray(np_contour_slice_255)

PNG.save("numpy_array_255.png")

GradientAnisotropicDiffusionImageFilter (0x1085b0ac0): Anisotropic diffusion unstable time step: 0.125
Stable time step for this image must be smaller than 0.0997431

GradientAnisotropicDiffusionImageFilter (0x1085b0ac0): Anisotropic diffusion unstable time step: 0.125
Stable time step for this image must be smaller than 0.0997431

GradientAnisotropicDiffusionImageFilter (0x1085b0ac0): Anisotropic diffusion unstable time step: 0.125
Stable time step for this image must be smaller than 0.0997431

GradientAnisotropicDiffusionImageFilter (0x1085b0ac0): Anisotropic diffusion unstable time step: 0.125
Stable time step for this image must be smaller than 0.0997431

GradientAnisotropicDiffusionImageFilter (0x1085b0ac0): Anisotropic diffusion unstable time step: 0.125
Stable time step for this image must be smaller than 0.0997431



# Get arc length.

`cv2.findContours` must be called on a `numpy` array. `cv2.findContours` returns the same result regardless of whether the array is binary (0|1) or binary (0|255). See unit test in [`test_imgproc.py`](../../tests/test_imgproc.py).

See notes on our [Wiki](https://github.com/COMP523TeamD/HeadCircumferenceTool/wiki/Dependencies).

In [11]:
# This returns the same result for binary arrays with 0's and 1's and binary arrays with 0's and 255's, see test_imgproc.py
# contours is a list of contours since there could be multiple contours. That is, contours[0] might be the outer contour and contours[1] might be the inner contour.
# There could be another contour contours[2] (or higher), depending on the image.
contours, hierarchy = cv2.findContours(
    np_contour_slice, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
)

print(f"Number of contours: {len(contours)}")
for i in range(len(contours)):
    print(f"\tArc length of contour {i}: {cv2.arcLength(contours[i], True)}")

Number of contours: 2
	Arc length of contour 0: 522.7005717754364
	Arc length of contour 1: 518.6000666618347


# `contours[0]` is always the parent contour, assuming there are no islands.

This is based on a unit test so may not be true for all cases.

See unit test in `test_imgproc.py`.

# Investigate one failing test case in `test_imgproc.test_arc_length_of_transposed_matrix_is_same_hardcoded`

The test passes for NIFTI_IMAGE (theta_x, theta_y, theta_z, slice_z) = (0, 0, 0, 0 to 144 with stepsize 18) but fails for (0, 0, 0, 162)

Slice 162 is an edge case and doesn't look like a brain slice. 34 contours are detected. It seems that the algorithm may not return the correct result for a transposed matrix that is an "edge case" (i.e. not a brain slice and lots of contours detected).

If Prof. Styner is okay with this, we might not need to be concerned about it. A user would never care about the circumference of an edge case like this. If it matters, then we can always transpose the matrix... or throw an error/warning when lots of contours are detected

In [12]:
reader = sitk.ImageFileReader()
reader.SetFileName(NIFTI_PATH)
image = reader.Execute()

sitk_contour: sitk.Image = rotate_and_get_contour(image, 0, 0, 0, 162)

np_contour_transposed = sitk.GetArrayFromImage(sitk_contour)
np_contour_not_transposed = np.ndarray.transpose(np_contour_transposed)

contours_transposed, hierarchy_transposed = cv2.findContours(
    np_contour_transposed, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
)
contours_not_transposed, hierarchy_not_transposed = cv2.findContours(
    np_contour_not_transposed, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
)

# Both of the first contours are the parent contour... not sure why the lengths are different...
assert hierarchy_transposed[0][0][3] == -1
assert hierarchy_not_transposed[0][0][3] == -1

print(f"Number of contours in transposed: {len(contours_transposed)}")
print(f"Number of contours in non-transposed: {len(contours_not_transposed)}")
print(
    f"Arc length of transposed parent contour: {cv2.arcLength(contours_transposed[0], True)}"
)
print(
    f"Arc length of non-transposed parent contour: {cv2.arcLength(contours_not_transposed[0], True)}"
)

Number of contours in transposed: 34
Number of contours in non-transposed: 34
Arc length of transposed parent contour: 7.656854152679443
Arc length of non-transposed parent contour: 10.485281229019165


GradientAnisotropicDiffusionImageFilter (0x1085b3530): Anisotropic diffusion unstable time step: 0.125
Stable time step for this image must be smaller than 0.0997431

GradientAnisotropicDiffusionImageFilter (0x1085b3530): Anisotropic diffusion unstable time step: 0.125
Stable time step for this image must be smaller than 0.0997431

GradientAnisotropicDiffusionImageFilter (0x1085b3530): Anisotropic diffusion unstable time step: 0.125
Stable time step for this image must be smaller than 0.0997431

GradientAnisotropicDiffusionImageFilter (0x1085b3530): Anisotropic diffusion unstable time step: 0.125
Stable time step for this image must be smaller than 0.0997431

GradientAnisotropicDiffusionImageFilter (0x1085b3530): Anisotropic diffusion unstable time step: 0.125
Stable time step for this image must be smaller than 0.0997431

